#  **_Applied Data Science Capstone_**
## Peer Graded Assignment:
## Segmenting and Clustering Neighborhoods in the city of Toronto, Canada


This file is to present the last part, the part 3, of the peer graded assignment. The first two parts are also accompanied at the outset to maintain the flow of the working process. 

## **_<span style="color: blue;">Please directly scroll down to Part 3: Explore, Cluster, and Visualize the neighborhoods in Toronto to grade the part 3.</span>_**


### <span style="color: blue;">Part 1: Transforming Wikipadia Table into Pandas Data Frame:</span>

> **_For this assignment, you will be required to explore and cluster the neighborhoods in Toronto._**

>   * Start by creating a new Notebook for this assignment.
>   * Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

> **_To create the dataframe:

>   * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
>   * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
>   * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
>   * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
>   * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
>   * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.



Download dependencies that I will use.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### <span style="color: blue;"> **_Scraping Wikipedia_** with Pandas </span>
    
Source: https://medium.com/swlh/how-to-scrape-a-website-with-a-single-line-of-python-code-5efe124275bb
    
> _"The Pandas library in Python includes a web scraper that pulls HTML table data into a dataframe in a single step. Simply insert the URL into the read_html() method and assign the resulting object to a variable so you can work with it. ... The catch is that this works only for text that has been put into table format in the underlying HTML. You can inspect the HTML to see if the data is in table form or you can just eyeball it. If it appears to be formatted in rows and columns, it’s likely to be table data. Wikipedia can be a great place to use the Pandas read_html()tool."_

In [2]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

Now, let's check the number of Table in the website.

In [3]:
print("# of Tables: ", len(data))

# of Tables:  3


Let's check 1) the shape and 2) the content of the first table.

In [4]:
print("Shape of the first Table: ", data[0].shape)
df=data[0]
df.head()

Shape of the first Table:  (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Yes, the first table is what we want.
Let's remove all the rows with 'Not assigned' in the column 'Neighbourhood'.

 * Here, we use **_Boolean Indexing_** to select all those assigned 'Borough'.

In [5]:
df_selected=df[df['Borough'] != 'Not assigned']
df_selected.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


> **_More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma_** 


df2 = df1.groupby(['Postcode','Borough'], sort=False).agg(', '.join)


In [6]:
df_selected_Grouped = df_selected.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df_selected_Grouped.reset_index(inplace=True)
print('type', type(df_selected_Grouped))
df_selected_Grouped.head() 

type <class 'pandas.core.frame.DataFrame'>


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

Accordingly, I will use np.where(condition, A, B) to replace 'Not assigned' in 'Neighbout' with 'Brough' name.
 * if the condition is satisfied, A is executed;
 * if the condition is breached, B is executed.

Source: https://numpy.org/doc/stable/reference/generated/numpy.where.html


In [7]:
df_selected_Grouped["Neighbourhood"] = np.where(df_selected_Grouped["Neighbourhood"]== "Not Assigned", df_selected_Grouped["Borough"], df_selected_Grouped["Neighbourhood"])
df_selected_Grouped.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now, let's check the shape of the cleaned dataframe

In [8]:
df_selected_Grouped.shape

(103, 3)

### <span style="color: blue;">Part 2: Integrating Geographical Coordinates into the Dataframe for Analysis:</span>

In Part 2, I will integrate the geographical coordinates into the dataframe that I have build.

> **_In order to utilize the Foursquare location data for analysis, we need to get the latitude and the longitude coordinates of each neighborhood. ... Here is the link to the csv file that has the geographical coordinates of each postal code: "Geospatial_Coordinates.csv" in the link, http://cocl.us/Geospatial_data_**

Now, I will download the geographical coordinates info from the link below:
https://cocl.us/Geospatial_data


In [9]:
!pip install wget
import wget
wget.download("https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv", "Geospatial_Coordinates.csv")


100% [................................................................................] 2891 / 2891

'Geospatial_Coordinates (4).csv'

In [10]:
coordinates_df= pd.read_csv("Geospatial_Coordinates.csv")
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now, I will merge the coordinates table and the Neighbourhood table that was cleaned above.

In [11]:
df_work = pd.merge(df_selected_Grouped, coordinates_df, on='Postal Code')
df_work.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### <span style="color: blue;">Part 3: Explore, Cluster, and Visualize the neighborhoods in Toronto:</span>

> **_Explore and cluster the Postal Code based neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto.           
... Just make sure:_**
>>* to add enough Markdown cells to explain what you decided to do and to report any observations you make.
>>* to generate maps to visualize your neighborhoods and how they cluster together. 

Now, I apply the analysis that was made with the New York City data in the lab to Toronto Data. 

Nevertheless, there is a fundamental difference between the Toronto dataframe for this assignment and the NY one for the lab: for the Toronto dataframe, we have one-to-one mapping between <span style="color: red;"> **_'Postal Code based Neighbours'_**</span> and the geographical code; while the NY counterpart, we build the one-to-one mapping between 'Neighbour' and the geographical code.

So, in this assignment, I will do <span style="color: red;"> **_'Postal Code based Neighbours'_**</span> analysis to explore and cluster.

### <span style="color: blue;"> **_Step 1) Dataframe Narrow-down_**</span>
    Now, let's extract Boroughs with 'Toronto' in their name.

In [12]:
df_work_Tronto=df_work[df_work["Borough"].str.contains('Toronto', regex=False)]
df_work_Tronto.reset_index(inplace=True, drop=True)
df_work_Tronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### <span style="color: blue;"> **_Step 2) Data Understanding_**</span>
   Now, let's use the first 'Postal Code' area to see what sort of data is included in Foursquare.
   
   First, let's check the geographical coordinates of the first 'Postal Code' Area.


In [13]:
PostalCodeArea_latitude = df_work_Tronto.loc[0, 'Latitude'] # neighborhood latitude value
PostalCodeArea_longitude = df_work_Tronto.loc[0, 'Longitude'] # neighborhood longitude value

PostalCodeArea_Post = df_work_Tronto.loc[0, 'Postal Code'] # neighborhood name per Postal Code
print('Latitude and longitude values of the Postal Code "{}" are {}, {}.'.format(PostalCodeArea_Post, 
                                                               PostalCodeArea_latitude, 
                                                               PostalCodeArea_longitude))

Latitude and longitude values of the Postal Code "M5A" are 43.6542599, -79.3606359.


OK, so, the first Postal Code is M5A.

Using the geographic coordinates above, I will send the request to explore the Foursquare info about the 'Postal Code' area of M5A. In order to do so, let me define Foursquare Credentials in a hidden cell.

### Define Foursquare Credentials and Version
The cell below contains sensitive information, personal credentials. Thus, it will be hidden using "# @hidden_cell.

In [14]:
# @hidden_cell
CLIENT_ID = 'YOUR_CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'YOUR_CLIENT_SECRET' # your Foursquare Secret
VERSION = 'YYYYMMDD' # Foursquare API version
#ACCESS_TOKEN = "?" Fill the "?"


In [15]:
LIMIT=100
Radius=500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, PostalCodeArea_latitude, PostalCodeArea_longitude, VERSION, Radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f20ba063757857343a034c4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

### <span style="color: blue;"> **_Step 3) Retrieving the Relevant Data from Foursquare_**</span>
    
Next, based on our findings on the Foursquare's venue information regarding the Postal Code, M5A, we can start retrieving venue information for all the Postal Code Areas in the dataframe. 

In the next cell, I will define the function that explores each 'Postal Code' area, using Foresquare API. 

In [16]:
def getNearbyVenues(Postal, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postal, lat, lng in zip(Postal, latitudes, longitudes):
        print(Postal)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now, let me pass the information from the Toronto dataframe that I prepared for the analysis into the function above.

In [17]:
Toronto_venues = getNearbyVenues(Postal=df_work_Tronto['Postal Code'], 
                                 latitudes=df_work_Tronto['Latitude'], 
                                 longitudes=df_work_Tronto['Longitude'])

0     M5A
1     M7A
2     M5B
3     M5C
4     M4E
5     M5E
6     M5G
7     M6G
8     M5H
9     M6H
10    M5J
11    M6J
12    M4K
13    M5K
14    M6K
15    M4L
16    M5L
17    M4M
18    M4N
19    M5N
20    M4P
21    M5P
22    M6P
23    M4R
24    M5R
25    M6R
26    M4S
27    M5S
28    M6S
29    M4T
30    M5T
31    M4V
32    M5V
33    M4W
34    M5W
35    M4X
36    M5X
37    M4Y
38    M7Y
Name: Postal Code, dtype: object
0     M5A
1     M7A
2     M5B
3     M5C
4     M4E
5     M5E
6     M5G
7     M6G
8     M5H
9     M6H
10    M5J
11    M6J
12    M4K
13    M5K
14    M6K
15    M4L
16    M5L
17    M4M
18    M4N
19    M5N
20    M4P
21    M5P
22    M6P
23    M4R
24    M5R
25    M6R
26    M4S
27    M5S
28    M6S
29    M4T
30    M5T
31    M4V
32    M5V
33    M4W
34    M5W
35    M4X
36    M5X
37    M4Y
38    M7Y
Name: Postal Code, dtype: object
0     M5A
1     M7A
2     M5B
3     M5C
4     M4E
5     M5E
6     M5G
7     M6G
8     M5H
9     M6H
10    M5J
11    M6J
12    M4K
13    M5K
14    M6K
15  

Let me check:
> 1. The size of the Venue List
2. The top 5 lines of the Toronto Venue info that we retrieved from Foursquare

In [18]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1642, 7)


,Postal Code,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let me summarise how many venues were returned for each Postal Code

In [19]:
Toronto_venues.groupby('Postal Code').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M4E,4,4,4,4,4,4
M4K,42,42,42,42,42,42
M4L,20,20,20,20,20,20
M4M,40,40,40,40,40,40
M4N,3,3,3,3,3,3
M4P,7,7,7,7,7,7
M4R,20,20,20,20,20,20
M4S,35,35,35,35,35,35
M4T,4,4,4,4,4,4


 Let's find out how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


### <span style="color: blue;"> **_Step 4) Analysis on Business Category_**</span>

Now, let me analyze the venue's business categories for each Postal Code area.

First, using 'one hot coding', I will prepare a new dataframe to analyze the venue's business category. 

In [21]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Postal Code'] = Toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

print("The size of the venue category matrix: ", Toronto_onehot.shape)
Toronto_onehot.head()

The size of the venue category matrix:  (1642, 236)


,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,

Next, let's group rows by 'Postal Code' and by taking the mean of the frequency of occurrence of each category¶

In [22]:
Toronto_grouped = Toronto_onehot.groupby('Postal Code').mean().reset_index()
Toronto_grouped

,Postal Code,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00

Check the shape of the new dataframe

In [23]:
num_top_venues = 5

for postal in Toronto_grouped['Postal Code']:
    print("----"+postal+"----")
    temp = Toronto_grouped[Toronto_grouped['Postal Code'] == postal].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0       Neighborhood  0.25
1  Health Food Store  0.25
2                Pub  0.25
3              Trail  0.25
4             Museum  0.00


----M4K----
                    venue  freq
0        Greek Restaurant  0.19
1             Coffee Shop  0.07
2      Italian Restaurant  0.07
3  Furniture / Home Store  0.05
4          Ice Cream Shop  0.05


----M4L----
              venue  freq
0               Gym  0.05
1  Sushi Restaurant  0.05
2              Park  0.05
3       Pizza Place  0.05
4               Pub  0.05


----M4M----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2              Brewery  0.05
3            Gastropub  0.05
4  American Restaurant  0.05


----M4N----
               venue  freq
0               Park  0.33
1           Bus Line  0.33
2        Swim School  0.33
3  Afghan Restaurant  0.00
4      Movie Theater  0.00


----M4P----
                  venue  freq
0  Gym / Fitness Center  0.14
1        Bre

Let's put that into a *pandas* dataframe
> 1. Define the function to sort the venue in descending order
2. Create a Dataframe to list the top 10 venues for each Postal Code.

In [24]:
# 1. Define the function to sort the venue in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
# 2. Create a Dataframe to list the top 10 venues for each Postal Code.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Postal_venues_sorted = pd.DataFrame(columns=columns)
Postal_venues_sorted['Postal Code'] = Toronto_grouped['Postal Code']

for ind in np.arange(Toronto_grouped.shape[0]):
    Postal_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Postal_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Trail,Neighborhood,Health Food Store,Pub,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio
1,M4K,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Indian Restaurant,Pub,Spa,Caribbean Restaurant
2,M4L,Gym,Coffee Shop,Brewery,Burrito Place,Sandwich Place,Restaurant,Pub,Pizza Place,Pet Store,Movie Theater
3,M4M,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Neighborhood,Seafood Restaurant,Sandwich Place,Cheese Shop
4,M4N,Park,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


###  <span style="color: blue;"> **_Step 5) Cluster Postal Code based Neighbours_** </span>
Run *k*-means to cluster the Postal Code based Neighbours into 5 clusters.

In [26]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 0, 2, 0, 0, 0, 3, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [27]:
# add clustering labels
Postal_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_work_Tronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Postal_venues_sorted.set_index('Postal Code'), on='Postal Code')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,Creperie,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Bubble Tea Shop,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Lingerie Store,Fast Food Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Restaurant,Clothing Store,Cosmetics Shop,Cocktail Bar,American Restaurant,Gym,Hotel,Breakfast Spot
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Neighborhood,Health Food Store,Pub,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio


Summarize the cluster distribution


In [28]:
Toronto_merged.groupby("Cluster Labels").count()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


###  <span style="color: blue;"> **_Step 6) Visualize the Cluster_** </span>
Finally, let's visualize the resulting clusters

In [29]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Postal Code'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## List Up Clusters
Here, I will list up the Postal Code based Neighbourhoods for each Cluster.


* The First Cluster List (Clister Label = 0).

In [30]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[0] +[1]+ [2] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Diner,Yoga Studio,Creperie,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",0,Coffee Shop,Clothing Store,Bubble Tea Shop,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Lingerie Store,Fast Food Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,0,Café,Coffee Shop,Restaurant,Clothing Store,Cosmetics Shop,Cocktail Bar,American Restaurant,Gym,Hotel,Breakfast Spot
5,M5E,Downtown Toronto,Berczy Park,0,Coffee Shop,Cocktail Bar,Farmers Market,Restaurant,Beer Bar,Café,Cheese Shop,Bakery,Seafood Restaurant,Hotel
6,M5G,Downtown Toronto,Central Bay Street,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Department Store,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Yoga Studio
7,M6G,Downtown Toronto,Christie,0,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Candy Store,Baby Store,Restaurant,Coffee Shop
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Thai Restaurant,Bar,Steakhouse,Gym,American Restaurant
9,M6H,West Toronto,"Dufferin, Dovercourt Village",0,Bakery,Pharmacy,Gym,Supermarket,Bar,Middle Eastern Restaurant,Café,Music Venue,Bank,Brewery
10,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",0,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery,Music Venue,Baseball Stadium


* The Second Cluster List.  (Clister Label = 1)

In [31]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[0] +[1]+ [2] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5N,Central Toronto,Roselawn,1,Garden,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


* The Third Cluster List (Clister Label = 2)

In [32]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[0] +[1]+ [2] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M4N,Central Toronto,Lawrence Park,2,Park,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


* The Fourth Cluster List (Clister Label = 3)

In [33]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[0] +[1]+ [2] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",3,Park,Jewelry Store,Trail,Sushi Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
29,M4T,Central Toronto,"Moore Park, Summerhill East",3,Park,Trail,Tennis Court,Restaurant,Colombian Restaurant,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
33,M4W,Downtown Toronto,Rosedale,3,Park,Playground,Trail,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


* Finally, the Fifth Cluster List (Clister Label = 4)

In [34]:
 Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[0] +[1]+ [2] + list(range(5, Toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4E,East Toronto,The Beaches,4,Trail,Neighborhood,Health Food Store,Pub,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio


### Overall Observation:

Overall, the venues are highly concentrated into the Cluster Label 0.